In [13]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error


In [14]:


class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

In [15]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [16]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [17]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [19]:
averaged_models = AveragingModels(models = (ENet, KRR, lasso))
averaged_models.fit(X_train, y_train)
pred = averaged_models.predict(X_test)
score = mean_absolute_error(y_test, pred)
print(" Averaged base models score: {:.4f}\n".format(score))

score = mean_absolute_error(y_test, np.expm1(pred))
print(" Averaged base models score: {:.4f}\n".format(score))
for x, y in zip(y_test, np.expm1(pred)):
    print(f"{x}\t{y}")

 Averaged base models score: 55.1807

 Averaged base models score: 32.1108

4	16.906790303746067
9	12.324617207537992
79	42.87787003281357
14	48.23156919538181
50	24.456807614277572
12	22.634245307754576
4	19.941159085636244
52	18.97248884143846
116	25.973780843125354
13	21.68989358627617
116	14.058570810979989
92	57.721817016636805
35	23.791349921212376
40	46.753581320115586
31	21.064378787166966
4	23.595939080112522
23	61.871630607652676
12	26.139917992618756
20	22.682324705382364
35	21.805232145963185
86	46.70764095208455
37	11.418680105763901
83	26.61644026330085
116	55.62647294701026
58	36.44706565261223
103	85.0279440494336
116	100.60963502053184
38	55.63509591022095
92	30.469960835948314
111	43.952190987465144
7	17.09708995118348
18	52.430185387181105
35	30.03716165537792
67	55.90263876028226
85	35.86921747343107
116	84.35767674822291
16	24.23854804489816
101	51.45331576500626
4	22.16064252833439
33	43.67315744874185
32	40.20436238290088
72	55.97270115798568
33	20.01246034030896